This Jupyter Notebook is used to algebraically calculate the derivative of the angular velocities $\dot{\omega}_i$. 

The same was done in the Mathematica Notebook *rattleback_equations_omega_dot.nb*, but the resulting equations where not quite as numerically stable. 

In [1]:
from sympy import *

# Define variables
m, g, h, alpha, beta = symbols('m g h alpha beta')
x1, x2, x3 = symbols('x1 x2 x3')
k1, k2, k3 = symbols('k1 k2 k3')

# ellipsoid parameters
a, b, c = symbols('a b c')
# abstand vom voll ellipsoidmittelpunkt zum schwerpunkt
h = symbols('h') 
# Trägheitstensor Einträge
A, B, C, F = symbols('A B C F')


# Luftreibungskoeffizienten
c1, c2, c3 = symbols('c_1 c_2 c_3')

w1, w2, w3 = symbols('omega1 omega2 omega3')

w1_dot = symbols(r"\dot{\omega}_1")
w2_dot = symbols(r"\dot{\omega}_2")
w3_dot = symbols(r"\dot{\omega}_3")


# Drehmomente
N1 = m * (-w1_dot * (x2**2 + (x3 + h)**2) + x1 * (w2_dot * x2 + w3_dot * (x3 + h))
          + k3 * x2 - k2 * (x3 + h) - g * ((x3 + h) * sin(alpha) - x2 * cos(alpha) * cos(beta)))

N2 = m * (-w2_dot * (x1**2 + (x3 + h)**2) + x2 * (w1_dot * x1 + w3_dot * (x3 + h))
          + k1 * (x3 + h) - k3 * x1 - g * cos(alpha) * (x1 * cos(beta) + (x3 + h) * sin(beta)))

N3 = m * (-w3_dot * (x1**2 + x2**2) + (x3 + h) * (w1_dot * x1 + w2_dot * x2)
          + k2 * x1 - k1 * x2 + g * (x1 * sin(alpha) + x2 * cos(alpha) * sin(beta)))

NR1 = - c1 * w1
NR2 = - c2 * w2
NR3 = - c3 * w3

In [2]:
# define the equations
eq_17_1 = Eq(N1 + NR1, A * w1_dot - F * w2_dot - (B - C) * w2 * w3 + F * w1 * w3)
eq_17_2 = Eq(N2 + NR2, B * w2_dot - F * w1_dot - (C - A) * w1 * w3 - F * w2 * w3) 
eq_17_3 = Eq(N3 + NR3, C * w3_dot - (A - B) * w1 * w2 + F * (w2**2 - w1**2))

In [3]:
#otherwise it takes too long, so first solve eq_17_3 for w3_dot
solution_w3_dot = solve(eq_17_3, w3_dot)
solution_w3_dot[0] # = w3_dot

(A*omega1*omega2 - B*omega1*omega2 + F*omega1**2 - F*omega2**2 + \dot{\omega}_1*h*m*x1 + \dot{\omega}_1*m*x1*x3 + \dot{\omega}_2*h*m*x2 + \dot{\omega}_2*m*x2*x3 - c_3*omega3 + g*m*x1*sin(alpha) + g*m*x2*sin(beta)*cos(alpha) - k1*m*x2 + k2*m*x1)/(C + m*x1**2 + m*x2**2)

In [4]:
# now substitute w3_dot in eq_17_1 and eq_17_2
eq_17_1_new = eq_17_1.subs(w3_dot, solution_w3_dot[0])
eq_17_2_new = eq_17_2.subs(w3_dot, solution_w3_dot[0])
# and solve for w1_dot and w2_dot - takes about 4.5 minutes
solution_eq_17 = solve([eq_17_1_new, eq_17_2_new], [w1_dot, w2_dot])

In [5]:
# simplyfy the solutions for w1_dot and w2_dot - takes about 2 minutes
w1_dot_eq = simplify(solution_eq_17[w1_dot])
w2_dot_eq = simplify(solution_eq_17[w2_dot])

In [6]:
w1_dot_string =str(w1_dot_eq).replace("**", "^")
print(w1_dot_string)

(A*B*h*m*omega1*omega2*x1 + A*B*m*omega1*omega2*x1*x3 - A*C*F*omega1*omega3 - A*C*m*omega1*omega3*x1*x2 + A*F*h*m*omega1*omega2*x2 + A*F*m*omega1*omega2*x2*x3 - A*F*m*omega1*omega3*x1^2 - A*F*m*omega1*omega3*x2^2 + A*h^3*m^2*omega1*omega2*x1 + 3*A*h^2*m^2*omega1*omega2*x1*x3 - A*h^2*m^2*omega1*omega3*x1*x2 + A*h*m^2*omega1*omega2*x1^3 + A*h*m^2*omega1*omega2*x1*x2^2 + 3*A*h*m^2*omega1*omega2*x1*x3^2 - 2*A*h*m^2*omega1*omega3*x1*x2*x3 + A*m^2*omega1*omega2*x1^3*x3 + A*m^2*omega1*omega2*x1*x2^2*x3 + A*m^2*omega1*omega2*x1*x3^3 - A*m^2*omega1*omega3*x1^3*x2 - A*m^2*omega1*omega3*x1*x2^3 - A*m^2*omega1*omega3*x1*x2*x3^2 + B^2*C*omega2*omega3 - B^2*h*m*omega1*omega2*x1 - B^2*m*omega1*omega2*x1*x3 + B^2*m*omega2*omega3*x1^2 + B^2*m*omega2*omega3*x2^2 - B*C^2*omega2*omega3 - B*C*F*omega1*omega3 - B*C*c_1*omega1 - B*C*g*h*m*sin(alpha) + B*C*g*m*x2*cos(alpha)*cos(beta) - B*C*g*m*x3*sin(alpha) + B*C*h^2*m*omega2*omega3 - B*C*h*k2*m + 2*B*C*h*m*omega2*omega3*x3 - B*C*k2*m*x3 + B*C*k3*m*x2 - B*C*m

In [7]:
w2_dot_string =str(w2_dot_eq).replace("**", "^")
print(w2_dot_string)

(-A^2*C*omega1*omega3 + A^2*h*m*omega1*omega2*x2 + A^2*m*omega1*omega2*x2*x3 - A^2*m*omega1*omega3*x1^2 - A^2*m*omega1*omega3*x2^2 - A*B*h*m*omega1*omega2*x2 - A*B*m*omega1*omega2*x2*x3 + A*C^2*omega1*omega3 + A*C*F*omega2*omega3 - A*C*c_2*omega2 - A*C*g*h*m*sin(beta)*cos(alpha) - A*C*g*m*x1*cos(alpha)*cos(beta) - A*C*g*m*x3*sin(beta)*cos(alpha) - A*C*h^2*m*omega1*omega3 + A*C*h*k1*m - 2*A*C*h*m*omega1*omega3*x3 + A*C*k1*m*x3 - A*C*k3*m*x1 + A*C*m*omega1*omega3*x1^2 - A*C*m*omega1*omega3*x3^2 + A*F*h*m*omega1^2*x2 + A*F*h*m*omega1*omega2*x1 - A*F*h*m*omega2^2*x2 + A*F*m*omega1^2*x2*x3 + A*F*m*omega1*omega2*x1*x3 - A*F*m*omega2^2*x2*x3 + A*F*m*omega2*omega3*x1^2 + A*F*m*omega2*omega3*x2^2 - A*c_2*m*omega2*x1^2 - A*c_2*m*omega2*x2^2 - A*c_3*h*m*omega3*x2 - A*c_3*m*omega3*x2*x3 - A*g*h*m^2*x1^2*sin(beta)*cos(alpha) + A*g*h*m^2*x1*x2*sin(alpha) - A*g*m^2*x1^3*cos(alpha)*cos(beta) - A*g*m^2*x1^2*x3*sin(beta)*cos(alpha) - A*g*m^2*x1*x2^2*cos(alpha)*cos(beta) + A*g*m^2*x1*x2*x3*sin(alpha) + A

In [8]:
# calculate w3_dot in dependens of w1_dot and w2_dot 
w3_dot_eq = solution_w3_dot[0].subs(w1_dot, symbols("omega1_dot")).subs(w2_dot, symbols("omega2_dot"))
w3_dot_string =str(w3_dot_eq).replace("**", "^")
print(w3_dot_string)

(A*omega1*omega2 - B*omega1*omega2 + F*omega1^2 - F*omega2^2 - c_3*omega3 + g*m*x1*sin(alpha) + g*m*x2*sin(beta)*cos(alpha) + h*m*omega1_dot*x1 + h*m*omega2_dot*x2 - k1*m*x2 + k2*m*x1 + m*omega1_dot*x1*x3 + m*omega2_dot*x2*x3)/(C + m*x1^2 + m*x2^2)


In [9]:
# this was used to check if the system of equations going from the derivative of the angles to derivative of the angular velocities (from Wackelstein_1.pdf) is correct
# it is not concerned with the calculation of the derivative of the angular velocities

eq1 = Eq((-w1*sin(beta)+w3*cos(beta))/cos(alpha), symbols(r"\dot{\gamma}"))
eq2 = Eq(w1*cos(beta)+w3*sin(beta), symbols(r"\dot{\alpha}"))
eq3 = Eq(w2 + (w1*sin(beta) - w3*cos(beta))*tan(alpha), symbols(r"\dot{\beta}"))

solution = solve([eq1, eq2, eq3], [w1, w2, w3])
simplify(solution[w3])


# checks out!

\dot{\alpha}*sin(beta) + \dot{\gamma}*cos(alpha)*cos(beta)